# PurpleAir PM Forecasted Interpolation

## Set Up

### Import Packages

In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
from datetime import date

### Set Workspace

In [2]:
cwd = os.getcwd()

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay
current_date = str(dt.datetime.today().date())
save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
boundary_path = os.path.join(cwd, '..', '..', 'data', 'mpls_boundary.geojson')
seasonal_path = historic_path = os.path.join(cwd, '..', '..', 'data', 'Seasonal_Predictions_'+ current_date + '.csv')
yearround_path = historic_path = os.path.join(cwd, '..', '..', 'data', 'Yearround_Predictions_' + current_date + '.csv')

In [3]:
output_fc = 'mpls_boundary'
output_fc_path = os.path.join(save_path_gdb, output_fc)

# Convert the GeoJSON file to a feature class
arcpy.JSONToFeatures_conversion(boundary_path, output_fc)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input JSON or GeoJSON: Dataset C:\Users\tande\Documents\GitHub\QualityAirQualityCities\arcpy\interpolation\..\..\data\mpls_boundary.geojson does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset C:\Users\tande\Documents\GitHub\QualityAirQualityCities\arcpy\interpolation\..\..\data\QAQC.gdb\mpls_boundary already exists.
Failed to execute (JSONToFeatures).


## Creating Points from Lat/Longs

In [3]:
# Create the feature class name
feature_class_name_seasonal = "Seasonal_" + current_date
feature_class_name_yearround = "Yearround_" + current_date

output_point_path_seasonal = os.path.join(save_path_gdb, feature_class_name_seasonal)
output_point_path_yearround = os.path.join(save_path_gdb, feature_class_name_yearround)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
station_points_seasonal = arcpy.management.XYTableToPoint(yearround_path, output_point_path_seasonal, x_field, y_field)
station_points_yearround = arcpy.management.XYTableToPoint(seasonal_path, output_point_path_yearround, x_field, y_field)

## PM 2.5 Interpolation

### Seasonal

#### Kriging

In [4]:
out_kriging_raster_seasonal = "Kriging_" + feature_class_name_seasonal
arcpy.ga.EmpiricalBayesianKriging(
    in_features=feature_class_name_seasonal,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_kriging_stats_" + feature_class_name_seasonal,
    out_raster=out_kriging_raster_seasonal,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'pm2_5_kriging_stats_Seasonal_May10'>

#### IDW

In [5]:
out_idw_raster_seasonal = "IDW_" + feature_class_name_seasonal
arcpy.ga.IDW(
    in_features=feature_class_name_seasonal,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_idw_stats_" + feature_class_name_seasonal,
    out_raster=out_idw_raster_seasonal,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'pm2_5_idw_stats_Seasonal_May10'>

#### Local Polynomial

In [6]:
out_poly_raster_seasonal = "Poly_" + feature_class_name_seasonal
arcpy.ga.LocalPolynomialInterpolation(
    in_features=feature_class_name_seasonal,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_poly_stats_" + feature_class_name_seasonal,
    out_raster=out_poly_raster_seasonal,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'pm2_5_poly_stats_Seasonal_May10'>

### Year-Round

#### Kriging

In [7]:
out_kriging_raster_yearround = "Kriging_" + feature_class_name_yearround
arcpy.ga.EmpiricalBayesianKriging(
    in_features=feature_class_name_yearround,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_kriging_stats_" + feature_class_name_yearround,
    out_raster=out_kriging_raster_yearround,
    cell_size=0.000932919999999967,
    transformation_type="NONE",
    max_local_points=100,
    overlap_factor=1,
    number_semivariograms=100,
    search_neighborhood="NBRTYPE=StandardCircular RADIUS=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    output_type="PREDICTION",
    quantile_value=0.5,
    threshold_type="EXCEED",
    probability_threshold=None,
    semivariogram_model_type="POWER"
)

<Result 'pm2_5_kriging_stats_Yearround_May10'>

#### IDW

In [8]:
out_idw_raster_yearround = "IDW_" + feature_class_name_yearround
arcpy.ga.IDW(
    in_features=feature_class_name_yearround,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_idw_stats_" + feature_class_name_yearround,
    out_raster=out_idw_raster_yearround,
    cell_size=10,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'pm2_5_idw_stats_Yearround_May10'>

#### Local Polynomial

In [9]:
out_poly_raster_yearround = "Poly_" + feature_class_name_yearround
arcpy.ga.LocalPolynomialInterpolation(
    in_features=feature_class_name_yearround,
    z_field="pm25_fullDay_mean_Prediction",
    out_ga_layer="pm2_5_poly_stats_" + feature_class_name_yearround,
    out_raster=out_poly_raster_yearround,
    cell_size=0.000932919999999967,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=9.87111293434023E-02 S_MINOR=9.87111293434023E-02 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result 'pm2_5_poly_stats_Yearround_May10'>

## Cross Validation

### Seasonal

#### Kriging PM 2.5

In [10]:
#cross validation
pm2_5_kriging_fc_seasonal = "pm2_5_kriging_cross_valid_" + feature_class_name_seasonal
#pm2_5_kriging_path_all = os.path.join(save_path_gdb, pm2_5_kriging_fc_all)
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_kriging_stats_" + feature_class_name_seasonal,
    out_point_feature_class=pm2_5_kriging_fc_seasonal
)
# add errors to shared PD DF
input_fc ="pm2_5_kriging_cross_valid_" + feature_class_name_seasonal
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
rmse_df = df.rename(columns={'Error': 'PM2_5_Kriging_Seasonal_Error'})

#### IDW PM 2.5

In [11]:
#cross validation
pm2_5_idw_fc_seasonal = "pm2_5_idw_cross_valid_" + feature_class_name_seasonal
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_idw_stats_" + feature_class_name_seasonal,
    out_point_feature_class=pm2_5_idw_fc_seasonal
)
# add errors to shared PD DF
input_fc ="pm2_5_idw_cross_valid_" + feature_class_name_seasonal
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_IDW_Seasonal_Error'})

#### Local Polynomial PM 2.5

In [12]:
#cross validation
pm2_5_poly_fc_seasonal = "pm2_5_poly_cross_valid_" + feature_class_name_seasonal
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_poly_stats_" + feature_class_name_seasonal,
    out_point_feature_class=pm2_5_poly_fc_seasonal
)
# add errors to shared PD DF
input_fc ="pm2_5_poly_cross_valid_" + feature_class_name_seasonal
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_Poly_Seasonal_Error'})

### Year-Round

#### Kriging PM 2.5

In [13]:
#cross validation
pm2_5_kriging_fc_yearround = "pm2_5_kriging_cross_valid_" + feature_class_name_yearround
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_kriging_stats_" + feature_class_name_yearround,
    out_point_feature_class=pm2_5_kriging_fc_yearround
)
# add errors to shared PD DF
input_fc ="pm2_5_kriging_cross_valid_" + feature_class_name_yearround
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_Kriging_Yearround_Error'})

#### IDW PM 2.5

In [14]:
#cross validation
pm2_5_idw_fc_yearround = "pm2_5_idw_cross_valid_" + feature_class_name_yearround
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_idw_stats_" + feature_class_name_yearround,
    out_point_feature_class=pm2_5_idw_fc_yearround
)
# add errors to shared PD DF
input_fc ="pm2_5_idw_cross_valid_" + feature_class_name_yearround
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_IDW_Yearround_Error'})

#### Local Polynomial 

In [15]:
#cross validation
pm2_5_poly_fc_yearround = "pm2_5_poly_cross_valid_" + feature_class_name_yearround
arcpy.ga.CrossValidation(
    in_geostat_layer="pm2_5_poly_stats_" + feature_class_name_yearround,
    out_point_feature_class=pm2_5_poly_fc_yearround
)
# add errors to shared PD DF
input_fc ="pm2_5_poly_cross_valid_" + feature_class_name_yearround
fields = ["OBJECTID", "Error"]

# Convert the feature class to a NumPy array and create a Pandas DataFrame
arr = arcpy.da.TableToNumPyArray(input_fc, fields)
df = pd.DataFrame(arr)
merged_df = rmse_df.merge(df, on='OBJECTID', how='left')
rmse_df = merged_df.rename(columns={'Error': 'PM2_5_Poly_Yearround_Error'})

In [14]:
rmse_df

,OBJECTID,PM2_5_Kriging_Seasonal_Error,PM2_5_IDW_Seasonal_Error,PM2_5_Poly_Seasonal_Error,PM2_5_Kriging_Yearround_Error,PM2_5_IDW_Yearround_Error,PM2_5_Poly_Yearround_Error
0,1,-1.897155,-2.784704,-1.680961,-1.557266,-1.720019,-1.401819
1,2,0.382091,0.259186,0.349050,0.315241,0.181062,0.194358
2,3,-0.325254,0.362930,0.235161,-0.272964,0.191443,-0.223475
3,4,-0.341965,0.014134,-0.230851,-0.376574,-0.272711,-0.482055
4,5,1.581623,2.811643,1.933801,0.546349,1.587447,0.748047
5,6,1.306676,1.435120,1.444577,1.092695,1.047549,1.115366
6,7,1.084602,1.200844,1.220804,0.584609,0.570917,0.545736
7,8,1.767244,1.351314,1.143959,-0.436536,-0.348884,-0.503800
8,9,0.198866,0.989185,0.009155,-0.347973,0.468901,-0.459708
9,10,0.553803,0.585852,0.230043,0.698926,0.716571,0.822317


## Calculate RMSE

### Seasonal

In [16]:
#calculate RMSE for each cross valid error
rmse_df['SQ_PM2_5_Kriging_Seasonal_Error'] = rmse_df['PM2_5_Kriging_Seasonal_Error'] ** 2
pm2_5_kriging_seasonal_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Kriging_Seasonal_Error'].mean())

In [17]:
rmse_df['SQ_PM2_5_IDW_Seasonal_Error'] = rmse_df['PM2_5_IDW_Seasonal_Error'] ** 2
pm2_5_idw_seasonal_rmse = numpy.sqrt((rmse_df['SQ_PM2_5_IDW_Seasonal_Error'].values).mean())

In [18]:
rmse_df['SQ_PM2_5_Poly_Seasonal_Error'] = rmse_df['PM2_5_Poly_Seasonal_Error'] ** 2
pm2_5_poly_seasonal_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Poly_Seasonal_Error'].mean())

In [28]:
# Create a dictionary to map the method names to the feature class names
method_feature_class_mapping_seasonal_interp = {
    'kriging': "Kriging_" + feature_class_name_seasonal,
    'idw': "IDW_" + feature_class_name_seasonal,
    'local polynomial': "Poly_" + feature_class_name_seasonal
}

method_feature_class_mapping_seasonal_cross_valid = {
    'kriging': "pm2_5_kriging_cross_valid_" + feature_class_name_seasonal,
    'idw': "pm2_5_idw_cross_valid_" + feature_class_name_seasonal,
    'local polynomial': "pm2_5_poly_cross_valid_" + feature_class_name_seasonal
}

# Compare all of the RMSEs to find the best
pm2_5_compare_seasonal = pd.DataFrame({
    'method': ['kriging', 'idw', 'local polynomial'],
    'RMSE': [pm2_5_kriging_seasonal_rmse, pm2_5_idw_seasonal_rmse, pm2_5_poly_seasonal_rmse]
})

# Get index of row with lowest value of RMSE
min_index_seasonal = pm2_5_compare_seasonal['RMSE'].idxmin()

# Get method with lowest RMSE
min_id_seasonal = pm2_5_compare_seasonal.loc[min_index_seasonal, 'method']

# Get the corresponding feature class name from the mapping dictionary
feature_class_name_seasonal_interp = method_feature_class_mapping_seasonal_interp[min_id_seasonal]
feature_class_name_seasonal_cross_valid = method_feature_class_mapping_seasonal_cross_valid[min_id_seasonal]

### Year-Round

In [32]:
#calculate RMSE for each cross valid error
rmse_df['SQ_PM2_5_Kriging_Yearround_Error'] = rmse_df['PM2_5_Kriging_Yearround_Error'] ** 2
pm2_5_kriging_yearround_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Kriging_Yearround_Error'].mean())

In [33]:
rmse_df['SQ_PM2_5_IDW_Yearround_Error'] = rmse_df['PM2_5_IDW_Yearround_Error'] ** 2
pm2_5_idw_yearround_rmse = numpy.sqrt((rmse_df['SQ_PM2_5_IDW_Yearround_Error'].values).mean())

In [34]:
rmse_df['SQ_PM2_5_Poly_Yearround_Error'] = rmse_df['PM2_5_Poly_Yearround_Error'] ** 2
pm2_5_poly_yearround_rmse = numpy.sqrt(rmse_df['SQ_PM2_5_Poly_Yearround_Error'].mean())

In [36]:
# Create a dictionary to map the method names to the feature class names
method_feature_class_mapping_yearround_interp = {
    'kriging': "Kriging_" + feature_class_name_yearround,
    'idw': "IDW_" + feature_class_name_yearround,
    'local polynomial': "Poly_" + feature_class_name_yearround
}

method_feature_class_mapping_yearround_cross_valid = {
    'kriging': "pm2_5_kriging_cross_valid_" + feature_class_name_yearround,
    'idw': "pm2_5_idw_cross_valid_" + feature_class_name_yearround,
    'local polynomial': "pm2_5_poly_cross_valid_" + feature_class_name_yearround
}

# Compare all of the RMSEs to find the best
pm2_5_compare_yearround = pd.DataFrame({
    'method': ['kriging', 'idw', 'local polynomial'],
    'RMSE': [pm2_5_kriging_yearround_rmse, pm2_5_idw_yearround_rmse, pm2_5_poly_yearround_rmse]
})

# Get index of row with lowest value of RMSE
min_index_yearround = pm2_5_compare_yearround['RMSE'].idxmin()

# Get method with lowest RMSE
min_id_yearround = pm2_5_compare_yearround.loc[min_index_yearround, 'method']

# Get the corresponding feature class name from the mapping dictionary
feature_class_name_yearround_interp = method_feature_class_mapping_yearround_interp[min_id_seasonal]
feature_class_name_yearround_cross_valid = method_feature_class_mapping_yearround_cross_valid[min_id_seasonal]

## Raster to Point Each Interpolation

### Seasonal

In [29]:
out_point_seasonal = "Raster_to_Point_" + feature_class_name_seasonal
arcpy.conversion.RasterToPoint(
    in_raster=feature_class_name_seasonal_interp,
    out_point_features=out_point_seasonal,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Raster_to_Point_Seasonal_May10'>

### Year-Round

In [37]:
out_point_yearround = "Raster_to_Point_" + feature_class_name_yearround
arcpy.conversion.RasterToPoint(
    in_raster=feature_class_name_yearround_interp,
    out_point_features=out_point_yearround,
    raster_field="Value"
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\Raster_to_Point_Yearround_May10'>

## Clipping to the Extent

### Seasonal

In [21]:
out_clip_seasonal = "PM2_5_Clipped_" + feature_class_name_seasonal
arcpy.analysis.Clip(
    in_features="Raster_to_Point_" + feature_class_name_seasonal,
    clip_features='mpls_boundary',
    out_feature_class=out_clip_seasonal,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Clipped_Seasonal_May10'>

### Year-Round

In [38]:
out_clip_yearround = "PM2_5_Clipped_" + feature_class_name_yearround
arcpy.analysis.Clip(
    in_features="Raster_to_Point_" + feature_class_name_yearround,
    clip_features='mpls_boundary',
    out_feature_class=out_clip_yearround,
    cluster_tolerance=None
)

<Result 'C:\\Users\\tande\\Documents\\GitHub\\QualityAirQualityCities\\arcpy\\interpolation\\..\\..\\data\\QAQC.gdb\\PM2_5_Clipped_Yearround_May10'>

## Save to Local and Remote Databases

### Connect to DB

In [26]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

try:
    conn = psycopg2.connect(**pg_connection_dict)
    print("connected")
except:
    print("connection failed")

connected


### Create & Populate Tables

#### Seasonal Interpolation

In [23]:
points = os.path.join(save_path_gdb, "PM2_5_Clipped_" + feature_class_name_seasonal)
fields_points = ['pointid', 'grid_code', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_Seasonal_Forecast")
cursor.execute("""
    CREATE TABLE PM2_5_Seasonal_Forecast (
        id SERIAL,
        pointid INT,
        grid_code DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_Seasonal_Forecast (pointid, grid_code, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

KeyboardInterrupt: 

#### Seasonal Cross Validation

In [30]:
points = os.path.join(save_path_gdb, feature_class_name_seasonal_cross_valid)
fields_points = ['OBJECTID','Error', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_Seasonal_Errors_Forecast")
cursor.execute("""
    CREATE TABLE PM2_5_Seasonal_Errors_Forecast (
        id SERIAL,
        objectid INT,
        error DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_Seasonal_Errors_Forecast (objectid, error, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

#### Year-Round Interpolation

In [39]:
points = os.path.join(save_path_gdb, "PM2_5_Clipped_" + feature_class_name_yearround)
fields_points = ['pointid', 'grid_code', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_YearRound_Forecast")
cursor.execute("""
    CREATE TABLE PM2_5_YearRound_Forecast (
        id SERIAL,
        pointid INT,
        grid_code DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_YearRound_Forecast (pointid, grid_code, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()

KeyboardInterrupt: 

#### Year-Round Cross Validation

In [40]:
points = os.path.join(save_path_gdb, feature_class_name_yearround_cross_valid)
fields_points = ['OBJECTID','Error', "SHAPE@WKT"]

# Create SQL table
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS PM2_5_Yearround_Errors_Forecast")
cursor.execute("""
    CREATE TABLE PM2_5_Yearround_Errors_Forecast (
        id SERIAL,
        objectid INT,
        error DOUBLE PRECISION,
        geometry geometry
        )
""")
conn.commit()

# Populate PostGIS
with arcpy.da.SearchCursor(points, fields_points) as da_cursor:
    for row in da_cursor:
        wkt = row[2]
        cursor.execute("INSERT INTO PM2_5_Yearround_Errors_Forecast (objectid, error, geometry) VALUES (%s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], wkt))
        conn.commit()